# Добавляем # slots:, если его нет. Работает, если есть # intent:
В строке `process_conll_file('<Ваш файл>', 'output.conll')` замените `<Ваш файл>` на название своего файла. Можно указать путь к файлу, но тогда надо напсать так `process_conll_file(r'<Ваш файл>', 'output.conll')`. В конце у вас должен пояится файл `'output.conll'`, где будут `# slots:`. Этот файл можно переименовать.


In [ ]:
import re


with open('ru.test_id.conll (1).txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# Замена строк с английским текстом
updated_content = re.sub(r'(?m)^# text: (.*)', r'# text EN: \1', file_content)


with open('ru.test_id.conll (1).txt', 'w', encoding='utf-8') as file:
    file.write(updated_content)

In [ ]:
def add_annotator_line(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    new_lines = []
    for line in lines:
        if line.startswith("# text EN:"):
            new_lines.append("# annotator: Zaryana\n") # любое имя аннотатора зависит от того, кто размечал
        new_lines.append(line)

    with open(filename, 'w', encoding='utf-8') as file:
        file.writelines(new_lines)

add_annotator_line("ru.test_id.conll (1).txt")


In [ ]:
def process_conll_file(input_filename, output_filename):
    with open(input_filename, "r") as f:
        read = f.read()
    lines = read.split('\n')
    output_lines = []
    found_intent = False  # Флаг для отслеживания строки intent

    # Проходим по каждой строке
    for i, line in enumerate(lines):
        # Добавляем текущую строку в список выходных строк
        output_lines.append(line)

        # Проверяем, начинается ли строка с "# intent:"
        if line.startswith('# intent:'):
            # Проверяем наличие строки # slots: после intent
            if (i + 1) < len(lines) and not lines[i + 1].startswith('# slots:'):
                # Добавляем строку # slots:
                output_lines.append('# slots:')

        # Если строка "# slots:" уже встретилась, сбрасываем флаг
        if line.startswith('# slots:'):
            found_intent = False

    # Объединяем список выходных строк в один текст
    output_text = '\n'.join(output_lines)

    # Записываем измененный текст в выходной файл
    with open(output_filename, "a") as f:
        f.write(output_text)

# Добавьте свой файл
process_conll_file('ru.test_id.conll (1) (2).txt', 'ru.test_id_132.conll')


# Проверяем на Табуляцию. Это надо сделать обязательно, потому что в следуюем коде я буду делить строку по табуляции и так вытаскивать слоты



In [ ]:
# Открываем файл для чтения и последующей записи
with open('ru.test (3).conll', 'r+', encoding='utf-8') as f:
    # Читаем содержимое файла
    content = f.read()
    # Разделяем содержимое файла на абзацы по пустой строке
    paragraphs = content.strip().split('\n\n')

    # Список для хранения обновленных абзацев
    updated_paragraphs = []

    # Проходим по каждому абзацу
    for paragraph in paragraphs:
        # Разделяем абзац на строки
        lines = paragraph.strip().split('\n')
        # Найдем индекс строки # slots:
        slots_line_index = -1
        for i, line in enumerate(lines):
            if line.startswith('# slots:'):
                slots_line_index = i
                break

        # Если нашли строку # slots:
        if slots_line_index != -1:
            # Начинаем проверку строк после строки # slots:
            for j in range(slots_line_index + 1, len(lines)):
                line = lines[j]
                # Разделяем строку по пробелам
                elements = line.split()
                # Если строка была разделена, то есть пробелы
                if len(elements) > 1:
                    # Объединяем элементы с табуляцией
                    tabbed_line = '\t'.join(elements)
                    # Обновляем строку с табуляцией
                    lines[j] = tabbed_line

        # Добавляем обновленный абзац в список
        updated_paragraphs.append('\n'.join(lines))

    # Переходим в начало файла и записываем обновленные абзацы
    f.seek(0)
    f.write('\n\n'.join(updated_paragraphs))
    f.truncate()  # Обрезаем остаток содержимого файла, если он есть


# Заполняем # slots. Тут два варианта первый для татарского чуть ниже для русского.
Добавьте название своего файла в `with open('<Ваш файл>', 'r+') as f:`

# Это вариант для татарского. Ниже будет для русского

In [ ]:
# Открываем файл для чтения и последующей записи
with open('<Ваш файл>', 'r+') as f:
    # Читаем содержимое файла
    read = f.read()
    # Разделяем содержимое файла на абзацы по пустой строке
    paragraphs = read.strip().split('\n\n')

    # Список для хранения обновленных абзацев
    updated_paragraphs = []

    # Проходим по каждому абзацу
    for paragraph in paragraphs:
        # Инициализируем список для хранения слотов
        slots = []
        # Разделяем абзац на строки
        lines = paragraph.strip().split('\n')
        # Получаем текст TAT из абзаца
        tat_line = [line for line in lines if line.startswith('# text TAT:')][0]
        tat_text = tat_line.split(':', 1)[1].strip()  # Извлекаем текст после '# text TAT:'

        # Начальные значения для отслеживания текущего слота
        current_slot_start = None
        current_slot_end = None
        current_mark = None

        # Проходим по строкам абзаца
        for line in lines:
            # Игнорируем строки, которые не содержат данные о словах
            if line.startswith('#'):
                continue

            # Разделяем строку на элементы: индекс, слово, intent, mark
            parts = line.split('\t')
            if len(parts) != 4:
                continue

            index, word, intent, mark = parts
            mark = mark.strip()

            # Проверяем метку слова
            if mark.startswith('O'):
                # Если встретили 'O' и у нас есть текущий слот, завершаем его
                if current_slot_start is not None and current_slot_end is not None:
                    slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')
                    current_slot_start = None
                    current_slot_end = None
                    current_mark = None
            else:
                # Извлекаем тип сущности из метки (удаляем префикс 'B-' или 'I-')
                entity_type = mark[2:]

                # Найдем начальный индекс слова в тексте TAT
                word_index = tat_text.find(word)

                if mark.startswith('B-'):
                    # Если слово начинается с 'B-', начинаем новый слот
                    if current_slot_start is not None and current_slot_end is not None:
                        slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')
                    current_slot_start = word_index + 1  # Увеличиваем индекс на 1 для начала
                    current_slot_end = word_index + len(word)
                    current_mark = entity_type
                elif mark.startswith('I-') and current_mark == entity_type:
                    # Если слово начинается с 'I-' и совпадает с текущей меткой, продолжаем текущий слот
                    current_slot_end = word_index + len(word)

        # Если у нас есть незавершенный слот после прохождения абзаца, добавляем его
        if current_slot_start is not None and current_slot_end is not None:
            slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')

        # Формируем строку для поля # slots
        slots_str = ', '.join(slots)

        # Проверяем, есть ли уже строка # slots в абзаце
        found_slots_line = False
        updated_lines = []
        for line in lines:
            if line.startswith('# slots:'):
                # Если нашли строку # slots:, заменяем её на новую строку slots_str
                updated_lines.append(f'# slots: {slots_str}')
                found_slots_line = True
            else:
                updated_lines.append(line)

        # Если строки # slots не было, добавляем её в конец абзаца
        if not found_slots_line:
            updated_lines.append(f'# slots: {slots_str}')

        # Добавляем обновленный абзац в список
        updated_paragraphs.append('\n'.join(updated_lines))

    # Переходим в начало файла и записываем обновленные абзацы
    f.seek(0)
    f.write('\n\n'.join(updated_paragraphs))
    f.truncate()  # Удаляем остаток содержимого файла, если оно есть

# Это вариант для русского

In [ ]:
import logging

# Настройка логирования
logging.basicConfig(filename='error.log', level=logging.ERROR, format='%(asctime)s %(levelname)s: %(message)s')

# Открываем файл для чтения и последующей записи
with open('ru.test (3).conll', 'r+', encoding='utf-8') as f:
    # Читаем содержимое файла
    read = f.read()
    # Разделяем содержимое файла на абзацы по пустой строке
    paragraphs = read.strip().split('\n\n')

    # Список для хранения обновленных абзацев
    updated_paragraphs = []

    # Проходим по каждому абзацу
    for paragraph in paragraphs:
        try:
            # Инициализируем список для хранения слотов
            slots = []
            # Разделяем абзац на строки
            lines = paragraph.strip().split('\n')
            # Получаем текст RU из абзаца
            ru_line = [line for line in lines if line.startswith('# text RU:') or line.startswith('# test RU:')][0]
            ru_text = ru_line.split(':', 1)[1].strip()  # Извлекаем текст после '# text RU:'

            # Начальные значения для отслеживания текущего слота
            current_slot_start = None
            current_slot_end = None
            current_mark = None

            # Проходим по строкам абзаца
            for line in lines:
                # Игнорируем строки, которые не содержат данные о словах
                if line.startswith('#'):
                    continue

                # Разделяем строку на элементы: индекс, слово, intent, mark
                parts = line.split('\t')
                if len(parts) != 4:
                    continue

                index, word, intent, mark = parts
                mark = mark.strip()

                # Проверяем метку слова
                if mark.startswith('O'):
                    # Если встретили 'O' и у нас есть текущий слот, завершаем его
                    if current_slot_start is not None and current_slot_end is not None:
                        slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')
                        current_slot_start = None
                        current_slot_end = None
                        current_mark = None
                else:
                    # Извлекаем тип сущности из метки (удаляем префикс 'B-' или 'I-')
                    entity_type = mark[2:]

                    # Найдем начальный индекс слова в тексте RU
                    word_index = ru_text.find(word)

                    if mark.startswith('B-'):
                        # Если слово начинается с 'B-', начинаем новый слот
                        if current_slot_start is not None and current_slot_end is not None:
                            slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')
                        current_slot_start = word_index + 1  # Увеличиваем индекс на 1 для начала
                        current_slot_end = word_index + len(word)
                        current_mark = entity_type
                    elif mark.startswith('I-') and current_mark == entity_type:
                        # Если слово начинается с 'I-' и совпадает с текущей меткой, продолжаем текущий слот
                        current_slot_end = word_index + len(word)

            # Если у нас есть незавершенный слот после прохождения абзаца, добавляем его
            if current_slot_start is not None and current_slot_end is not None:
                slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')

            # Формируем строку для поля # slots
            slots_str = ', '.join(slots)

            # Проверяем, есть ли уже строка # slots в абзаце
            found_slots_line = False
            updated_lines = []
            for line in lines:
                if line.startswith('# slots:'):
                    # Если нашли строку # slots:, заменяем её на новую строку slots_str
                    updated_lines.append(f'# slots: {slots_str}')
                    found_slots_line = True
                else:
                    updated_lines.append(line)

            # Если строки # slots не было, добавляем её в конец абзаца
            if not found_slots_line:
                updated_lines.append(f'# slots: {slots_str}')

            # Добавляем обновленный абзац в список
            updated_paragraphs.append('\n'.join(updated_lines))

        except IndexError as e:
            # Логируем ошибку и абзац, вызвавший её, вместе с названием ошибки
            logging.error(f'Error processing paragraph: {paragraph}\nError: {e}')
            continue

    # Переходим в начало файла и записываем обновленные абзацы
    f.seek(0)
    f.write('\n\n'.join(updated_paragraphs))
    f.truncate()  # Удаляем остаток содержимого файла, если оно есть

